<h1 align="center"> Aplicações em Processamento de Linguagem Natural </h1>
<h2 align="center"> Aula 03 - Técnicas de Pré-Processamento de Texto e Similaridade</h2>
<h3 align="center"> Prof. Fernando Vieira da Silva MSc.</h3>

<h2> Técnicas para Pré-Processamento - Parte 2</h2>

<p>Uma vez que o texto já foi devidamente tratado, removendo stopwords e pontuações, e aplicando stemming ou lemmatization, agora precisamos contar a frequência das palavras (ou n-grams) que utilizaremos em seguida como atributos para as técnicas de aprendizado de máquina.</p>

<b>1. TF-IDF (Term Frequency - Inverse Document Frequency)</b>

<p><b>Term Frequency:</b> um termo que aparece muito em um documento, tende a ser um termo importante. Em resumo, divide-se o número de vezes em que um termo apareceu pelo maior número de vezes em que algum outro termo apareceu no documento.</p>

tf<sub>wd</sub> = f<sub>wd</sub> / m<sub>wd</sub>

onde:<br>
f<sub>wd</sub> é o número de vezes em que o termo <i>w</i> aparece no documento <i>d</i>.<br>
m<sub>wd</sub> é o maior valor de f<sub>wd</sub> obtido para algum termo do documento <i>d</i><br>

<p><b>Inverse Document Frequency:</b> um termo que aparece em poucos documentos pode ser um bom descriminante. Obtem-se dividindo o número de documentos pelo número de documentos em que o termo aparece.</p>

idf<sub>w</sub> = log<sub>2</sub>(n / n<sub>w</sub>)

onde:<br>
n é o número de documentos no corpus
n<sub>w</sub> é o número de documentos em que o termo <i>w</i> aparece.

Na prática, usa-se:

tf-idf = tf<sub>wd</sub> * idf<sub>w</sub>

Podemos calcular o TF-IDF de um corpus usando o pacote <b>scikit-learn</b>. Primeiramente, vamos abrir novamente o texto de Hamlet e armazenar as sentenças em uma ndarray do numpy (como se cada sentença fosse um documento do corpus):

In [ ]:
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize

hamlet_raw = nltk.corpus.gutenberg.raw('shakespeare-hamlet.txt')

sents = sent_tokenize(hamlet_raw)

hamlet_np = np.array(sents)

print(hamlet_np.shape)


<p>Agora vamos definir uma função para tokenização pelo scikit-learn.</p>

In [ ]:
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import wordnet

stopwords_list = stopwords.words('english')

lemmatizer = WordNetLemmatizer()

def my_tokenizer(doc):
    words = word_tokenize(doc)
    
    pos_tags = pos_tag(words)
    
    non_stopwords = [w for w in pos_tags if not w[0].lower() in stopwords_list]
    
    non_punctuation = [w for w in non_stopwords if not w[0] in string.punctuation]
    
    lemmas = []
    for w in non_punctuation:
        if w[1].startswith('J'):
            pos = wordnet.ADJ
        elif w[1].startswith('V'):
            pos = wordnet.VERB
        elif w[1].startswith('N'):
            pos = wordnet.NOUN
        elif w[1].startswith('R'):
            pos = wordnet.ADV
        else:
            pos = wordnet.NOUN
        
        lemmas.append(lemmatizer.lemmatize(w[0], pos))

    return lemmas
    
    

E essa função será chamada pelo objeto TfidfVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

hamlet_raw = nltk.corpus.gutenberg.raw('shakespeare-hamlet.txt')

sents = sent_tokenize(hamlet_raw)

hamlet_np = np.array(sents)

tfidf_vectorizer = TfidfVectorizer(tokenizer=my_tokenizer)

tfs = tfidf_vectorizer.fit_transform(hamlet_np)

print(tfs.shape)

In [ ]:
print([k for k in tfidf_vectorizer.vocabulary_.keys()][:20])

In [ ]:
print(tfs[:50,:50])

In [ ]:
import pprint
print(tfs[0,:60].toarray())
print(tfs[0].shape)

In [ ]:
for item in tfidf_vectorizer.vocabulary_.items():
    print(item)
    if item[1] == 17:
        print(item[0])

In [ ]:
print(tfs[0])

<b>2. TF-IDF de N-gramas</b>

Opcionalmente, podemos obter os atributos tf-idf de n-grams, combinando as classes CountVectorizer e TfidfTransformer. Em nosso exemplo, vamos utilizar apenas trigramas:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_vect = CountVectorizer(ngram_range=(3,3))

n_gram_counts = count_vect.fit_transform(hamlet_np)

tfidf_transformer = TfidfTransformer()

tfs_ngrams = tfidf_transformer.fit_transform(n_gram_counts)

print(tfs_ngrams.shape)

In [ ]:
for item in count_vect.vocabulary_.items():
    print(item)

In [ ]:
print(tfs_ngrams[0,:17].toarray())

<b>3. Redução de Dimensionalidade</b>

<p>A transformação do corpus em atributos contendo as frequências TF-IDF em geral resultará numa ndarray bastante esparsa, ou seja, com muitas dimensões. Porém, além de isso tornar o treinamento de algoritmos mais demorado e custoso (computacionalmente falando), muitas dessas dimensões provavelmente são pouco representativas ou mesmo podem causar ruído durante o treinamento. Para resolver esse problema, podemos aplicar uma técnica de redução de dimensionalidade simples chamada <b>Singular Value Decomposition (SVD)</b>. 

<p>Essa técnica transformará os vetores da matriz original, rotacionando e escalando-os, resultando em novas representações. A redução de dimensionalidade é feita ao manter apenas as <i>k</i> dimensões mais representativas que escolhermos. Outra vantagem dessa técnica é que as dimensões originais são, de certa forma, "combinadas", o que resulta em uma nova forma de representar a combinação de termos. No contexto de PLN, essa técnica é conhecida como <b>Latent Semantic Analysis (LSA)</b></p>

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd_transformer = TruncatedSVD(n_components=1000)

svd_transformer.fit(tfs)

print(sorted(svd_transformer.explained_variance_ratio_)[::-1][:30])

<p>Agora vamos manter as dimensões até que a variância acumulada seja maior ou igual a 0.50.</p>

In [ ]:
cummulative_variance = 0.0
k = 0
for var in sorted(svd_transformer.explained_variance_ratio_)[::-1]:
    cummulative_variance += var
    if cummulative_variance >= 0.70:
        break
    else:
        k += 1
        
print(k)

<p>Transformarmos novamente, mas desta vez com o número de k componentes que obtemos anteriormente.</p>

In [ ]:
svd_transformer = TruncatedSVD(n_components=k)
svd_data = svd_transformer.fit_transform(tfs)
print(sorted(svd_transformer.explained_variance_ratio_)[::-1])

In [ ]:
print(svd_data.shape)
print(svd_data)

<h2> Similaridade de Cosseno</h2>

<p>Uma vez que dois documentos são representados como vetores numéricos, é possível comparar a similaridade entre os documentos ao calcular o cosseno do ângulo entre esses documentos - uma medida de distância, não de amplitude. Para isso, basta resolver a equação do produto escalar entre os vetores, para encontrar o cosseno.</p>

<img src="http://s0.wp.com/latex.php?latex=++%5Cdisplaystyle++%5Cvec%7Ba%7D+%5Ccdot+%5Cvec%7Bb%7D+%3D+%5C%7C%5Cvec%7Ba%7D%5C%7C%5C%7C%5Cvec%7Bb%7D%5C%7C%5Ccos%7B%5Ctheta%7D+%5C%5C+%5C%5C++%5Ccos%7B%5Ctheta%7D+%3D+%5Cfrac%7B%5Cvec%7Ba%7D+%5Ccdot+%5Cvec%7Bb%7D%7D%7B%5C%7C%5Cvec%7Ba%7D%5C%7C%5C%7C%5Cvec%7Bb%7D%5C%7C%7D++&amp;bg=ffffff&amp;fg=000000&amp;s=0" alt="  \displaystyle  \vec{a} \cdot \vec{b} = \|\vec{a}\|\|\vec{b}\|\cos{\theta} \\ \\  \cos{\theta} = \frac{\vec{a} \cdot \vec{b}}{\|\vec{a}\|\|\vec{b}\|}  " title="  \displaystyle  \vec{a} \cdot \vec{b} = \|\vec{a}\|\|\vec{b}\|\cos{\theta} \\ \\  \cos{\theta} = \frac{\vec{a} \cdot \vec{b}}{\|\vec{a}\|\|\vec{b}\|}  " class="latex">

<img class=" wp-image-2582 " title="vector_space" src="http://blog.christianperone.com/wp-content/uploads/2013/09/vector_space.png" alt="" width="504" height="378">
<p fontsize=8>Fonte: http://blog.christianperone.com</p>

In [ ]:
    documents = (
    "The sky is blue",
    "The sun is bright",
    "The sun in the sky is bright",
    "We can see the shining sun, the bright sun"
    )

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)
print(tfidf_matrix.shape)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

query_vect = tfidf_vectorizer.transform(["The sun is red in the sky"])
#query_vect = tfidf_vectorizer.transform(["The sun in the sky is bright"])

cosine_similarity(query_vect, tfidf_matrix)

<p><b>Exercício 3:</b>Escreva um chatbot que, dado uma pergunta em Inglês, encontre uma pergunta mais parecida no corpus de perguntas e respostas disponível no Kaggle (https://www.kaggle.com/rtatman/questionanswer-dataset#S08_question_answer_pairs.txt) e exiba a pergunta original e a resposta.

In [ ]:
!ls ../input

In [ ]:
qa_file = open('../input/S08_question_answer_pairs.txt', 'r')

In [ ]:
qa_file.read()